In [8]:
# pip install keras
# pip install graphviz
# pip install ann_visualizer

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import keras
import ann_visualizer
import graphviz
import math
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [6]:
# Cargue del dataset

concrete = pd.read_csv(r'''D:\Camilo's Stuff\Javeriana\Concrete_Data_Yeh.csv''')
concrete.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   cement            1030 non-null   float64
 1   slag              1030 non-null   float64
 2   flyash            1030 non-null   float64
 3   water             1030 non-null   float64
 4   superplasticizer  1030 non-null   float64
 5   coarseaggregate   1030 non-null   float64
 6   fineaggregate     1030 non-null   float64
 7   age               1030 non-null   int64  
 8   csMPa             1030 non-null   float64
dtypes: float64(8), int64(1)
memory usage: 72.5 KB


In [7]:
concrete.head()

,cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,csMPa
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [14]:
# Separar features y target
features = concrete.drop('csMPa',axis=1)
target = concrete['csMPa'].values

In [16]:
#Separar dataset en train y test y escalar

X_train, X_test, y_train, y_test = train_test_split(features, targets, train_size=0.8, random_state=42)

scaler = StandardScaler().fit(X_train)
X_train_scaled = pd.DataFrame(scaler.transform(X_train), index=X_train.index.values, columns=X_train.columns.values)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), index=X_test.index.values, columns=X_test.columns.values)

In [17]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV 
from sklearn.metrics import r2_score
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt      


def training_model(X_train, y_train):
        model = DecisionTreeRegressor()

        # Grid Search CV
        parameters = {'max_depth'     : [4,6,8],
                      'criterion'     : ['mse', 'mae', 'friedman_mse'],
                      'splitter'      : ['best', 'random'],
                      'min_weight_fraction_leaf'  : [0.0,0.1,0.2,0.3],
                      'random_state' : [7,23,42,78,142],
                      'min_impurity_decrease' : [0.0,0.05,0.1,0.2]
                      # Add more parameters here for tuning
                      }
        grid = GridSearchCV(estimator=model, param_grid = parameters, cv = 10, 
                            verbose = 1, n_jobs = -1, refit = True)
        grid.fit(X_train, y_train)

        # Results from Grid Search
        print("\n========================================================")
        print(" Resultados de la Búsqueda por Grid " )
        print("========================================================")    
        print("\n El mejor estimador probando todos los parámetros evaluados:\n",
              grid.best_estimator_)
        print("\n Los mejores estimadores probando todos los parámetros evaluados:\n",
              grid.best_params_)
        print("\n ========================================================")

        # Random Search CV
        parameters = {'max_depth'     : sp_randInt(4, 10),
                      'criterion'     : ['mse', 'mae', 'friedman_mse'],
                      'splitter'      : ['best', 'random'],
                      'random_state': sp_randInt(4, 10),
                      'min_impurity_decrease'  : sp_randFloat(),
                      # Add more parameters here for tuning
                      }    
        randm = RandomizedSearchCV(estimator=model, 
                                   param_distributions = parameters, cv = 10, 
                                   n_iter = 10, verbose = 1, n_jobs = -1)
        randm.fit(X_train, y_train)

        # Results from Random Search
        print("\n========================================================")
        print(" Resultados de la Búsqueda Aleatoria " )
        print("========================================================")    
        print("\n El mejor estimador probando todos los parámetros evaluados: \n",
              randm.best_estimator_)
        print("\n El mejor score probando todos los parámetros evaluados:\n",
              randm.best_score_)
        print("\n Los mejores estimadores probando todos los parámetros evaluados:\n",
              randm.best_params_)
        print("\n ========================================================")
        print()

        print()
        print("Score de la Búsqueda aleatoria: ", randm.best_score_)
        print()
        print("Score de la Búsqueda por Grid: ", grid.best_score_)        
        print()

        if grid.best_score_ > randm.best_score_:
            print("El mejor modelo encontrado en la Búsqueda por Grid ... ... ... ...\n\n")
            return(grid.best_estimator_)
        else:
            print("El mejor modelo encontrado en la Búsqueda aleatoria ... ... ... ...\n\n")
            return(randm.best_estimator_)

model = training_model(X_train, y_train)

Fitting 10 folds for each of 1440 candidates, totalling 14400 fits

 Resultados de la Búsqueda por Grid 

 El mejor estimador probando todos los parámetros evaluados:
 DecisionTreeRegressor(max_depth=8, min_impurity_decrease=0.05, random_state=42)

 Los mejores estimadores probando todos los parámetros evaluados:
 {'criterion': 'mse', 'max_depth': 8, 'min_impurity_decrease': 0.05, 'min_weight_fraction_leaf': 0.0, 'random_state': 42, 'splitter': 'best'}

Fitting 10 folds for each of 10 candidates, totalling 100 fits

 Resultados de la Búsqueda Aleatoria 

 El mejor estimador probando todos los parámetros evaluados: 
 DecisionTreeRegressor(criterion='friedman_mse', max_depth=9,
                      min_impurity_decrease=0.6253019504393584, random_state=9)

 El mejor score probando todos los parámetros evaluados:
 0.8180676001262963

 Los mejores estimadores probando todos los parámetros evaluados:
 {'criterion': 'friedman_mse', 'max_depth': 9, 'min_impurity_decrease': 0.6253019504393584

In [21]:
from sklearn.metrics import r2_score
from scipy.stats import spearmanr, pearsonr

pred          = model.predict(X_test)
r2            = r2_score(y_test, pred)
spearman = spearmanr(y_test, pred)
pearson = pearsonr(y_test, pred)
print(f'Test data R-2 score: {r2:>5.3}')
print(f'Test data Spearman correlation: {spearman[0]:.3}')
print(f'Test data Pearson correlation: {pearson[0]:.3}')


Test data R-2 score: 0.789
Test data Spearman correlation: 0.883
Test data Pearson correlation: 0.89


In [22]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=500, oob_score=True, random_state=0)
rf.fit(X_train, y_train)

RandomForestRegressor(n_estimators=500, oob_score=True, random_state=0)

In [23]:
from sklearn.metrics import r2_score
from scipy.stats import spearmanr, pearsonr

predicted_train = rf.predict(X_train)
predicted_test = rf.predict(X_test)

test_score = r2_score(y_test, predicted_test)
spearman = spearmanr(y_test, predicted_test)
pearson = pearsonr(y_test, predicted_test)

print(f'Out-of-bag R-2 score estimate: {rf.oob_score_:>5.3}')
print(f'Test data R-2 score: {test_score:>5.3}')
print(f'Test data Spearman correlation: {spearman[0]:.3}')
print(f'Test data Pearson correlation: {pearson[0]:.3}')

Out-of-bag R-2 score estimate: 0.916
Test data R-2 score: 0.879
Test data Spearman correlation: 0.939
Test data Pearson correlation: 0.94


In [26]:
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import ParameterGrid


# Grid de hiperparámetros evaluados
# ==============================================================================
param_grid = {'n_estimators': [300, 500],
              'max_features': [3, 5, 7, 9],
              'max_depth'   : [None, 3, 10, 20]
             }

# Búsqueda por grid search con validación cruzada
# ==============================================================================
grid = GridSearchCV(
        estimator  = RandomForestRegressor(random_state = 123),
        param_grid = param_grid,
        scoring    = 'neg_root_mean_squared_error',
        cv         = RepeatedKFold(n_splits=5, n_repeats=3, random_state=123), 
        refit      = True,
        verbose    = 0,
        return_train_score = True
       )

grid.fit(X = X_train, y = y_train)

# Resultados
# ==============================================================================
resultados = pd.DataFrame(grid.cv_results_)
resultados.filter(regex = '(param.*|mean_t|std_t)') \
    .drop(columns = 'params') \
    .sort_values('mean_test_score', ascending = False) \
    .head(4)

D:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "D:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 387, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "D:\anaconda3\lib\site-packages\joblib\parallel.py", line 1041, in __call__
    if self.dispatch_one_batch(iterator):
  File "D:\anaconda3\lib\site-packages\joblib\parallel.py", line 859, in dispatch_one_batch
    self._dispatch(tasks)
  File "D:\anaconda3\lib\site-packages\joblib\parallel.py", line 777, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)
  File "D:\anaconda3\lib\site-packages\joblib\_parallel_backends.py", l

,param_max_depth,param_max_features,param_n_estimators,mean_test_score,std_test_score,mean_train_score,std_train_score
3,None,5,500,-5.304745,0.357655,-2.012975,0.093740
27,20,5,500,-5.306734,0.359187,-2.013299,0.093967
2,None,5,300,-5.314429,0.359046,-2.024361,0.089959
26,20,5,300,-5.317442,0.361844,-2.024787,0.090228


In [27]:
# Mejores hiperparámetros por validación cruzada
# ==============================================================================
print("----------------------------------------")
print("Mejores hiperparámetros encontrados (cv)")
print("----------------------------------------")
print(grid.best_params_, ":", grid.best_score_, grid.scoring)

----------------------------------------
Mejores hiperparámetros encontrados (cv)
----------------------------------------
{'max_depth': None, 'max_features': 5, 'n_estimators': 500} : -5.304744627183075 neg_root_mean_squared_error


In [28]:
# Error de test del modelo final
# ==============================================================================
from sklearn.metrics import mean_squared_error
modelo_final = grid.best_estimator_
predicciones = modelo_final.predict(X = X_test)
rmse = mean_squared_error(
        y_true  = y_test,
        y_pred  = predicciones,
        squared = False
       )
print(f"El error (rmse) de test es: {rmse}")

El error (rmse) de test es: 5.405848023499954


In [31]:
#predicted_train = rf.predict(X_train)

test_score = r2_score(y_test, predicciones)
spearman = spearmanr(y_test, predicciones)
pearson = pearsonr(y_test, predicciones)

print(f'Out-of-bag R-2 score estimate: {rf.oob_score_:>5.3}')
print(f'Test data R-2 score: {test_score:>5.3}')
print(f'Test data Spearman correlation: {spearman[0]:.3}')
print(f'Test data Pearson correlation: {pearson[0]:.3}')

Out-of-bag R-2 score estimate: 0.916
Test data R-2 score: 0.887
Test data Spearman correlation: 0.944
Test data Pearson correlation: 0.945


## BOOSTING

In [38]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV 
    
GBR = GradientBoostingRegressor()
            
parameters = {'learning_rate': [0.01, 0.05, 0.1, 0.13,0.15,0.2],
              'n_estimators' : [100, 500, 800,1000],
              'max_depth'    : [3,4,5 ,6, 8]
             }
    
grid_GBR = GridSearchCV(estimator=GBR, param_grid = parameters, cv = 5)
grid_GBR.fit(X_train, y_train)    

print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",grid_GBR.best_estimator_)
print("\n The best score across ALL searched params:\n",grid_GBR.best_score_)
print("\n The best parameters across ALL searched params:\n",grid_GBR.best_params_)

 Results from Grid Search 

 The best estimator across ALL searched params:
 GradientBoostingRegressor(n_estimators=800)

 The best score across ALL searched params:
 0.926934496077864

 The best parameters across ALL searched params:
 {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 800}


In [39]:
from sklearn.metrics import r2_score
from scipy.stats import spearmanr, pearsonr

modelo_final = grid_GBR.best_estimator_
predicciones = modelo_final.predict(X = X_test)

test_score = r2_score(y_test, predicciones)
spearman = spearmanr(y_test, predicciones)
pearson = pearsonr(y_test, predicciones)


print(f'Test data R-2 score: {test_score:>5.3}')
print(f'Test data Spearman correlation: {spearman[0]:.3}')
print(f'Test data Pearson correlation: {pearson[0]:.3}')

Test data R-2 score: 0.927
Test data Spearman correlation: 0.964
Test data Pearson correlation: 0.963


In [40]:
rmse = mean_squared_error(
        y_true  = y_test,
        y_pred  = predicciones,
        squared = False
       )
print(f"El error (rmse) de test es: {rmse}")

El error (rmse) de test es: 4.347941871355606
